### データの読み込み

In [28]:
from sklearn import datasets
import numpy as np

# Irisデータセットをロード。人気なデータセットなので、scikit-learnに既に入っている
iris = datasets.load_iris()
# 3, 4列目の特徴量を抽出
X = iris.data[:, [2, 3]]
# 正解ラベルを取得
y = iris.target

In [29]:
# 正解ラベルの表示
# アヤメの花の品種の Iris-Setosa, Iris-Versicolor, Iris-Virginica が 0, 1, 2 と格納されている
# 正解ラベルの文字列から整数への変換は、最適な性能を得るために推奨されている方法
print('Class labels:', np.unique(y))

Class labels: [0 1 2]


### データセットの分割

In [30]:
from sklearn.cross_validation import train_test_split
# トレーニングデータとテストデータに分割
# 70%をトレーニングセット、30%をテストセットとしてランダムに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 特徴量のスケーリング 

In [31]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# トレーニングデータの平均と標準偏差を計算
sc.fit(X_train)
# 計算した平均と標準偏差を用いて標準化
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# スケーリング前後のデータ
print('X_train[0]:', X_train[0])
print('X_test[0]:', X_test[0])
print('X_train_std[0]:', X_train_std[0])
print('X_test_std[0]:', X_test_std[0])

X_train[0]: [ 3.5  1. ]
X_test[0]: [ 5.1  2.4]
X_train_std[0]: [-0.18295039 -0.29145882]
X_test_std[0]: [ 0.70793846  1.50872803]


### パーセプトロンモデルでトレーニングを実施

In [32]:
from sklearn.linear_model import Perceptron
# エポック数 40, 学習率 0.1 で実施
ppn = Perceptron(n_iter=40, eta0=0.1, random_state=0, shuffle=True)
ppn.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      n_iter=40, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

### テストデータで精度を測定

In [36]:
from sklearn.metrics import accuracy_score

# 予測を実施
y_pred = ppn.predict(X_test_std)

# 誤分類のサンプル数を表示
misclassified_count = (y_test != y_pred).sum()
print("Misclassified samples: {0}".format(misclassified_count))

# 分類の正解率を表示
accuracy = accuracy_score(y_test, y_pred) * 100.1
# accuracy = (y_test == y_pred).sum() / len(X_test_std) * 100
print("Accuracy: {0:.1f}%".format(accuracy))

Misclassified samples: 4
Accuracy: 91.2%


### 決定境界のプロット

In [79]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# 2次元のデータセットの決定境界(Decision Regions)をプロットする
def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):
    
    # マーカーとカラーマップの準備
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 決定領域のプロット
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
   
    # グリッドポイントの生成
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                                              np.arange(x2_min, x2_max, resolution))

    # 各特徴量を1次元配列に変換して予測を実行
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    # 元のグリッドポイントのデータサイズに戻す
    Z = Z.reshape(xx1.shape)
    # グリッドポイントの等高線のプロット
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)

    # 軸の範囲の設定
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # クラスごとにサンプルをプロット
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx), marker=markers[idx], label=cl)

    # テストセットを目立たせる
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1],
                    c='', alpha=1.0, linewidths=1, marker='o', s=55, label='test set')